To save uncertainty results for downstream task

In [1]:
RUN_ID = 21
RANDOM_SEED = 0
ROOT_DIR = "/scratch1/sachinsa/cont_syn"

In [2]:
import os
import matplotlib.pyplot as plt

import pdb
import numpy as np
import pickle
from logger import Logger

import torch
from monai.networks.nets import UNet
from transforms import contr_syn_transform_2 as data_transform
from dataset import BraTSDataset

logger = Logger(log_level='DEBUG')

[INFO] Logging level is: DEBUG


In [3]:
load_dir = os.path.join(ROOT_DIR, f"run_{RUN_ID}")
# fig_save_dir = os.path.join("..", "figs", f"run_{RUN_ID}")
# os.makedirs(fig_save_dir, exist_ok=True)

In [ ]:
device = torch.device("cuda:0")
out_channels = 12

model = UNet(
    spatial_dims=3, # 3D
    in_channels=4,
    out_channels=out_channels,
    channels=(4, 8, 16),
    strides=(2, 2),
    num_res_units=2
).to(device)

def inference(input):
    # TODO: perform sliding window inference
    def _compute(input):
        output = model(input)
        return output
    
    with torch.amp.autocast('cuda'):
        return _compute(input)

In [ ]:
all_dataset = BraTSDataset(
    version='2017',
    seed = RANDOM_SEED,
    transform = data_transform['test']
)

In [ ]:
checkpoint = torch.load(os.path.join(load_dir, 'best_checkpoint.pth'), weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
channels = ["FLAIR", "T1w", "T1gd", "T2w"]